# Classification des images
                                            Emanuela Boros
                                            Université de La Rochelle

# 1. Introduction

La classification des objets consiste à attribuer une classe à un objet. 

<figure>
    <center>
    <img src="images/classification.jpeg" width="400" height="600">
    <figcaption>Source: <a href="https://www.freecodecamp.org/news/chihuahua-or-muffin-my-search-for-the-best-computer-vision-api-cbda4d6b425d/">Link</a></figcaption>
    </center>
</figure>

Dans ce cours, nous explorerons un ensemble de données, une analyse de texte et plusieurs approches basées sur l'apprentissage automatique et profond, de la manière suivante:

- **Récupération** du corpus d'images (*.csv, *.json, etc.)
- **Prétraitement** des images (image pre-processing) : réduction de taille etc
- **Representation** des images (pixels, HOG, SIFT)
- **Apprentissage automatique** (machine learning) et **apprentissage profond** (deep learning)
- **Analyse d'erreurs** (evaluation, etc.)

# 2. Récupération, Prétraitement et Exploration du corpus des images


## 2. 1. Contexte

Le gouvernement américain a attaqué en justice cinq grands groupes américains du tabac pour avoir amassé d'importants bénéfices en mentant sur les dangers de la cigarette. Le cigarettiers  se sont entendus dès 1953, pour "mener ensemble une vaste campagne de relations publiques afin de contrer les preuves de plus en plus manifestes d'un lien entre la consommation de tabac et des maladies graves". 

Dans ce procès 6,910,192 de documents ont été collectés et numérisés. Afin de faciliter l'exploitation de ces documents par les avocats, vous êtes en charge de mettre en place une classification automatique des types de documents: **Advertisement, Email, Form, Letter, Memo, News, Note, Report, Resume, Scientific**.


<table>
  <tr>
    <td>Letter</td>
     <td>News</td>
  </tr>
  <tr>
    <td><img src="images/50661869-1869.jpg" width=270 height=480></td>
    <td><img src="images/10031617.jpg" width=270 height=480></td>
  </tr>
 </table>


Un échantillon aléatoire des documents a été collecté et des opérateurs ont classé les documents dans des répertoires correspondant aux classes de documents : lettres, rapports, notes, email, etc. Vous avez à votre disposition : 

- le chemin de l'image; <ins>**path**</ins>
- les classes des documents définies par des opérateurs; <ins>**label**</ins>

Nous pourrons charger et regarder le structure de ces commentaires avec la bibliothèque pandas (plus d'info [```pandas```](https://pandas.pydata.org/docs/user_guide/index.html)).



In [ ]:
import pandas as pd # https://pandas.pydata.org/docs/

data = pd.read_csv('data/Tobacco3482-images/tobacco_images.csv')#.sample(frac=.5) # lire le fichier .csv

In [ ]:
len(data)

In [ ]:
data.head() # afficher les 5 premières lignes

In [ ]:
data.tail() # afficher les 5 dernières lignes

Avec Pandas, nous pouvons commencer avec quelques statistiques simples. Les résultats suivants seront dans l'ordre décroissant de sorte que le premier élément soit l'élément le plus fréquent.

In [ ]:
data.label.value_counts()

Avec Pandas, nous pouvons également visualiser ces statistiques.

In [ ]:
_ = data.label.value_counts().plot(kind='bar')

Nous avons vu que le fichier .csv contenait les chemins et les étiquettes des documents. Maintenant, nous devons lire les fichiers à partir des chemins spécifiés pour obtenir les images.


In [ ]:
!pip install opencv-python opencv-contrib-python

In [ ]:
import os
import numpy as np
import cv2

images = []
for idx, line in data.iterrows(): # itérer dans un dataframe Pandas
    image = cv2.imread(line['path'])
    images.append(image)
    
data['image'] = images # créer une nouvelle colonne dans le dataframe Pandas avec les textes associés

In [ ]:
data.head()

## 2.2 Pré-traitement des images 

Comme dans la première partie où nous avons représenté les mots par des vecteurs, nous allons représenter les images par des vecteurs aussi. Comme pour les mots, il y a plusieurs types de représentations pour les images. Chaque représentation peut ajouter des informations supplémentaires sur une image. Plus précisément, certains types de représentations fournissent des informations qui ne sont pas évidentes dans une représentation triviale.

Parmi les représentations existantes, nous allons analyser:
 - RGB
 - Gris
 - HOG

### RGB (Red, Green, Blue)

Rouge, vert, bleu, abrégé en RVB ou en RGB (de l'anglais « red, green, blue ») est un système de codage informatique des couleurs, le plus proche du matériel. Le codage RVB indique une valeur pour chacune de ces couleurs primaires. 

<figure>
    <center>
    <img src="images/sample_image.png" width="400" height="600">
    <figcaption>height, width, channels</figcaption>
    </center>
</figure>

In [ ]:
print('height, width, channels', data['image'][0].shape) # shape = height, width, channels

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.imshow(data['image'][0])
plt.show()

Puisque nos données sont grises, nous prenons une autre image comme exemple (pour afficher les valeurs RGB):

In [ ]:
import cv2
sample_image = cv2.imread('images/apples.jpg', cv2.COLOR_BGR2RGB)

def get_RGB_image(image):
    # converting image to HSV format with cv2.COLOR_BGR2RGB
    sample_rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return sample_rgb_image

sample_image = get_RGB_image(sample_image)

print("Image Properties")
print("Number of Pixels: " + str(sample_image.size))
print("Shape/Dimensions (height, width, channels): " + str(sample_image.shape))

In [ ]:
print("Pixels:", sample_image)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(sample_image)
plt.title('Original Image')
plt.show()

In [ ]:
image = cv2.resize(sample_image, (300, 300))

plt.imshow(image)
plt.title('Resized Image')
plt.show()

In [ ]:
# We separate the image in the 3 channels: red, green and blue
channel_initials = list('RGB')

for channel_index in range(3):
    
    channel = np.zeros(shape=sample_image.shape, dtype='uint8')
    channel[:,:,channel_index] = sample_image[:,:,channel_index]

    plt.imshow(channel)
    plt.title(channel_initials[channel_index] + ' channel')
    plt.show()

### Gris

Une autre possibilité est la représentation de ces images dans les nuances de gris. 

In [ ]:
def get_gray_image(image):

    # converting image to HSV format with cv2.COLOR_BGR2GRAY
    sample_gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return sample_gray_image

sample_gray_image = get_gray_image(sample_image)
plt.imshow(sample_gray_image, cmap='gray', vmin=0, vmax=255)
plt.title('Gray Image')
plt.show()

### HSV (hue, saturation, value)

HSV est une représentation alternative du modèle de couleur RGB. La représentation HSV modélise la façon dont les couleurs apparaissent sous la lumière.
Teinte (hue) spécifie l'angle de la couleur sur le cercle de couleur RGB.
La saturation contrôle la quantité de couleur utilisée.
La valeur (value) contrôle la luminosité de la couleur.

In [ ]:
def get_HSV_image(image):
    # converting image to HSV format with cv2.COLOR_BGR2HSV
    sample_hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    return sample_hsv_image

sample_hsv_image = get_gray_image(sample_image)
plt.imshow(sample_hsv_image)
plt.show()

In [ ]:
print("Pixels:", sample_hsv_image.shape)

Nous pouvons également transformer l'image en **différentes nuances de gris**.

In [ ]:
def get_gray_thresh_binary(image):
    _, thresh_binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    return thresh_binary

def get_gray_thresh_binary_inv(image):
    _, thresh_binary_inv = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
    return thresh_binary_inv
 
def get_gray_thresh_trunc(image):
    _, thresh_trunc = cv2.threshold(image, 127, 255, cv2.THRESH_TRUNC)
    return thresh_trunc
 
def get_gray_thresh_tozero(image):
    _, thresh_tozero = cv2.threshold(image, 127, 255, cv2.THRESH_TOZERO)
    return thresh_tozero
 
def get_gray_thresh_tozero_inv(image):
    _, thresh_tozero_inv = cv2.threshold(image, 127, 255, cv2.THRESH_TOZERO_INV)
    return thresh_tozero_inv

# Displaying the different thresholding styles
 
names = ['Original Image','BINARY','THRESH_BINARY','THRESH_TRUNC','THRESH_TOZERO','THRESH_TOZERO_INV']
 
images = [sample_gray_image, 
          get_gray_thresh_binary(sample_gray_image), 
          get_gray_thresh_binary_inv(sample_gray_image), 
          get_gray_thresh_trunc(sample_gray_image), 
          get_gray_thresh_tozero(sample_gray_image), 
          get_gray_thresh_tozero_inv(sample_gray_image)]

for i in range(6):
 
    plt.subplot(2, 3, i+1), plt.imshow(images[i], cmap='gray', vmin=0, vmax=255)
 
    plt.title(names[i])

## 2.3 Répresentation des images 


### Canny Edge Detection (détection de bord Canny) 

- est un algorithme de détection de bord populaire. Il a été développé par John F. Canny en 1986. C'est un algorithme en plusieurs étapes et nous passerons par plusieurs étapes. Deux des plus importants sont: la réduction du bruit (étant donné que la détection des contours est sensible au bruit dans l'image, la première étape consiste à supprimer le bruit dans l'image avec un filtre gaussien 5x5.) Et la recherche du gradient d'intensité de l'image.

In [ ]:
def get_edges(image):
    #calculate the edges using Canny edge algorithm
    edges_of_image = cv2.Canny(sample_gray_image, 100, 200) 
    return edges_of_image

plt.figure(figsize=(10, 10))

plt.imshow(sample_gray_image, cmap='gray')
plt.show()

#plot the edges
edges_of_image = get_edges(image)
plt.figure(figsize=(10, 10))
plt.imshow(edges_of_image, cmap='gray')
plt.show()

### Gaussian blur : réduire le bruit de l'image

Dans le traitement d'image, un flou gaussien (également connu sous le nom de lissage gaussien) est le résultat du flou d'une image par une fonction gaussienne (du nom du mathématicien et scientifique Carl Friedrich Gauss).

C'est un effet largement utilisé généralement pour réduire le bruit de l'image et réduire les détails.

In [ ]:
import numpy as np

plt.figure(figsize=(10, 10))
plt.imshow(sample_gray_image, cmap='gray')
plt.show()

def get_smoothen_image(image):
    #using the averaging kernel for image smoothening 
    averaging_kernel = np.ones((5,5), np.float32)/25
    filtered_image = cv2.filter2D(image, -1, averaging_kernel) 
    return filtered_image

#show smoothen image
filtered_image = get_smoothen_image(sample_gray_image)
plt.figure(figsize=(10, 10))
plt.imshow(filtered_image, cmap='gray') 
plt.show()

def get_gaussian_blur(image):
    #get a one dimensional Gaussian Kernel 
    gaussian_kernel_x = cv2.getGaussianKernel(15, 5) 
    gaussian_kernel_y = cv2.getGaussianKernel(15, 5) 

    #converting to two dimensional kernel using matrix multiplication 
    gaussian_kernel = gaussian_kernel_x * gaussian_kernel_y.T 

    #you can also use cv2.GaussianBLurring(image,(shape of kernel),standard deviation) instead of cv2.filter2D 
    filtered_image = cv2.filter2D(sample_gray_image, -1, gaussian_kernel) 
    return filtered_image


#show smoothen image
filtered_image = gaussian_blur(sample_gray_image)
plt.figure(figsize=(10, 10))
plt.imshow(filtered_image, cmap='gray') 
plt.show()

### SIFT (scale-invariant feature transform)

La transformation de caractéristiques invariantes d'échelle (SIFT) est un algorithme de détection de caractéristiques en vision par ordinateur pour détecter et décrire les caractéristiques locales dans les images.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

#create sift object
sift  = cv2.xfeatures2d.SIFT_create()

def get_SIFT_features(image):
    #calculate keypoints and their orientation
    keypoints, sift_image = sift.detectAndCompute(image, None)
    return sift_image


keypoints, sift_image = sift.detectAndCompute(sample_gray_image, None)
#plot keypoints on the image
image_with_keypoints = cv2.drawKeypoints(sample_gray_image, keypoints, sample_gray_image)

#plot the image
plt.figure(figsize=(10, 10))
plt.imshow(image_with_keypoints)
plt.show()

### Histogram of Oriented Gradients (HOG)

Un histogramme de gradient orienté (HOG) est une caractéristique utilisée en vision par ordinateur pour la détection d'objet. La technique calcule des histogrammes locaux de l'orientation du gradient sur une grille dense, c'est-à-dire sur des zones régulièrement réparties sur l'image. Cette méthode est particulièrement efficace pour la détection de personnes. Plus d'info <a href="https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html">ici</a>. Nous allons utiliser la bibliothèque <a href="https://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_rgb_to_gray.html">skimage</a>.

In [ ]:
!pip install scikit-image

In [ ]:
from skimage.feature import hog

def get_HOG_image(image):
    # Get HOG features
    hog_features, hog_image = hog(image,
                                  visualize=True,
                                  block_norm='L2-Hys',
                                  pixels_per_cell=(16, 16))
    return hog_image

plt.figure(figsize=(10, 10))
plt.imshow(sample_gray_image, cmap='gray')
plt.show()

hog_image = get_HOG_image(sample_gray_image)
plt.figure(figsize=(10, 10))
plt.imshow(hog_image, cmap='gray')
plt.show()

**Ensuite, nous créons une fonction pour prétraiter tous les images dans le dataframe Pandas.**


In [ ]:
print(data['image'][0].shape)
print(data['image'][0].flatten().shape)

In [ ]:
def preprocess(image):
    
    # resize image
    image = cv2.resize(image, (300, 300))
    
    # tranform image into grayscale
    image = get_gray_image(image)
    
    # we need to work with vectors so we transform the 3D matrix (height, width, channels) in 1D (a list) 
    image = image.flatten()
    
    return image

data['preprocessed_image'] = data['image'].apply(preprocess)

In [ ]:
data.head() # visualisez les donnees apres pre-traitement

# 3. Preparation pour apprentissage automatique (machine learning) et apprentissage profond (deep learning)

### 3.1 Diviser les jeux de données 
Afin d'entraîner des modèles pour apprentissage automatique et évaluer la performance de ses modèles avec chaque répresentation de mots, nous allons diviser les jeux de données en : entraînement, validation et test.

Pour plus de simplicité, nous allons travailler uniquement avec des données pour entraînement et test.

In [ ]:
from sklearn.model_selection import train_test_split

# Nous utilisons ici la colonne qui a été prétraitée.
X_train, X_test, y_train, y_test = train_test_split(list(data['preprocessed_image']), data['label'], test_size=0.1)
print('Train:', len(X_train), 'images', 'Test:', len(X_test), 'images')

# 4. Apprentissage automatique (machine learning)

L'objectif de la catégorisation de textes est de pouvoir associer automatiquement des documents à des classes (catégories, étiquettes, index) prédéfinies. Nous nous plaçons dans le cadre de l'apprentissage automatique supervisé. 



## 4.1 Principal component analysis (PCA)

L'analyse en composantes principales (ACP ou PCA en anglais pour principal component analysis) est une méthode de la famille de l'analyse des données et plus généralement de la statistique multivariée, qui consiste à transformer des variables liées entre elles en nouvelles variables décorrélées les unes des autres. 

Ces nouvelles variables sont nommées « composantes principales », ou axes principaux. Elle réduit le nombre de variables et rend l'information moins redondante sur une donnée. 

On utilise PCA pour extraire les composants les plus pertinents dans les vecteurs des images.



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# La normalisation des ensembles de données est une exigence 
# courante pour de nombreux estimateurs d'apprentissage automatique
# Avant d'appliquer PCA, il est nécessaire de standardiser vos données 
# (travailler avec des images signifie également travailler avec de grandes valeurs).
# Les algorithmes peuvent se comporter mal si les caractéristiques individuelles 
# ne ressemblent pas plus ou moins à des données standard normalement distribuées. 
# N'oubliez pas de normaliser vos données lorsque vous travaillez avec des images.
standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)

# Now, we apply PCA to reduce the vectors size
pca = PCA(n_components=50)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print('PCA train data matrix shape is: ', X_train.shape)
print('PCA test data matrix shape is: ', X_test.shape)

## 4.2 Approaches de classification

### La régression logistique 
- est un algorithme de classification qui transforme sa sortie à l'aide de la fonction sigmoïde logistique pour donner une valeur de probabilité pour les classes de sortie.


La bibliothèque sklearn fournit une classe [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) qui nous permet d'entraîner et tester un modèle à partir d'un jeux de données d'entraînement.

Ensuite, nous utilisons les données d'entraînement pour entraîner le modèle.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults
classifier = LogisticRegression()

_ = classifier.fit(list(X_train), y_train) # Nous entrainons le modèle avec la méthode `fit`
print('Training finished.')

Après avoir entraîné notre modèle, nous pouvons faire la prediction des classes des messages du jeux de données de test.
 
## Analyse d'erreurs

Le taux d'erreur de classification donne une évaluation des performances pour toutes les classes. Mais comme les classes ne sont pas également réparties, elles peuvent ne pas être également bien modélisées. Afin d'avoir une meilleure idée des performances du classifieur, des métriques détaillées doivent être utilisées:

* [metrics.classification_report](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) fournit une analyse détaillée par classe: la précision (parmi tous les exemples classés en classe X, combien sont réellement de la classeX) et le rappel (parmi tous les exemples qui sont de la classe X, combien sont classés en classe X) et le F-Score qui est une moyenne harmonique pondérée de la précision et du rappel.

F1-score = 2 x $\frac{précision\ x\ rappel}{précision\ +\ rappel}$

 - précision : la proportion d'identifications positives était effectivement correcte.
 - rappel : la proportion de résultats positifs réels a été identifiée correctement. 


In [ ]:
from sklearn.metrics import classification_report

y_pred = classifier.predict(list(X_test))

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(confusion_matrix(y_test, y_pred), annot=True)
plt.ylabel('True classes')
plt.xlabel('Predicted classes')

plt.show()

### SVM

Les machines à vecteurs de support (en anglais support vector machine, SVM) sont un ensemble de techniques d'apprentissage supervisé destinées à résoudre des problèmes de discrimination et de régression. Ces techniques reposent sur deux idées clés : la notion de marge maximale et la notion de fonction noyau.

La bibliothèque sklearn propose un module pour utiliser de [svm.SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). 

**<span style="color:red">To do</span>**:

> * Entraînez, testez et faites l'évaluation du modèle SVM linéaire. Essayez également le SVM avec différents noyaux (kernel) : `poly`, `rbf`, `sigmoid`. Pour le SVM linéaire, c'est `SVC(kernel='linear')`.
> * Rapportez le `classification_report` pour votre classificateur. Quelles classes ont les meilleurs scores ? Pourquoi ?


In [ ]:
from sklearn.svm import SVC

# YOUR CODE HERE

print('Training finished.')

In [ ]:
from sklearn.metrics import classification_report


## **<span style="color:red">To do</span>**:

Les trois représentations de base dans ce cours sont:
 - RGB
 - Gris
 - HSV
 - détection de bord Canny
 - HOG
 - SIFT

Nous pouvons utiliser une représentation toute seule ou faire de combinaisons pour ajouter caractéristiques supplémentaires qui sont possibles avec différentes représentations. Dans ce cadre, nous pouvons essayer:
 - RGB
 - Gris
 - RGB, Canny
 - Gris, HOG
 - RGB, HOG, Canny
 - SIFT, gris
 - et d'autres combinaison


> * Utilisez les autres méthodes de prétraitement (avec lemmatisation, avec racinisation, etc. `get_RGB_image, get_gray_image, get_HSV_image, get_gray_thresh_binary, get_gray_thresh_binary_inv, get_gray_thresh_trunc, get_gray_thresh_tozero, 
get_gray_thresh_tozero_inv, get_edges, get_smoothen_image, get_gaussian_blur, get_SIFT_features`) et comparez les résultats de ces trois modèles. Vous devez réécrire le code à partir de "5.1 Diviser les jeux de données" et changer la colonne de données. **Lorsque vous utilisez `get_SIFT_features`, appliquez` image = image.flatten()[:1024]`. La sortie de `get_SIFT_features` a des vecteurs de taille différente, nous prendrons un maximum de 1024 valeurs.**
> * Réécrivez la méthode de prétraitement avec différentes combinaisons, divisez l'ensemble de données, réduisez la taille avec StandardScaler et PCA entraîne les modèles de régression logistique et SVM. Vérifiez la precision (accuracy et F1-score) des résultats. 


In [ ]:
def preprocess(image):
    
    # resize image
    image = cv2.resize(image, (300, 300))
    
    # YOUR CODE HERE
    
    # we need to work with vectors so we transform the 3D matrix (height, width, channels) in 1D (a list) 
    image = image.flatten()
    
    return image

data['preprocessed_image'] = data['image'].apply(preprocess)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Nous utilisons ici la colonne qui a été prétraitée.
X_train, X_test, y_train, y_test = train_test_split(list(data['preprocessed_image']), data['label'], test_size=0.1)
print('Train:', len(X_train), 'images', 'Test:', len(X_test), 'images')

standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)

pca = PCA(n_components=50)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print('PCA train data matrix shape is: ', X_train.shape)
print('PCA test data matrix shape is: ', X_test.shape)

In [220]:
# YOUR CODE HERE

In [221]:
# YOUR CODE HERE

In [222]:
# YOUR CODE HERE

In [223]:
# YOUR CODE HERE

In [224]:
# YOUR CODE HERE

In [225]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

# 5. Apprentissage profond (deep learning)

## Classification avec des réseaux de neurones

Les réseaux de neurones peuvent être entraînés pour apprendre à la fois la représentation vectorielle des mots (au lieu de tf-idf) et comment classer les documents. Le code ci-dessous vous permet d'entraîner un classificateur des images neuronal convolutif. La plupart du code est écrit, il suffit de définir l'architecture du réseau avec les bons paramètres avant de l'entraîner:

## **<span style="color:red">To do</span>**:

> * Aller plus loin. Vérifiez [Image classification from scratch](https://keras.io/examples/vision/image_classification_from_scratch/)
> * Comment ce réseau de neurones se compare-t-il aux autres modèles? 
> * Quelle est la performance?
> * Qu'apporte le changement de paramètres dans la performance?
> * Utilisez des plongements pré-entraînés et chargez-les en tant que poids dans ce modèle (au lieu de ceux générés aléatoirement)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Pour appliquer des méthodes d'apprentissage en profondeur, pour les images, nous travaillerons avec deux dossiers (`train` et `test`) avec les données qui étaient déjà divisées auparavant. Nous utiliserons la méthode `image_dataset_from_directory` pour charger les images pour l'entraînement et tester le modèle.

In [ ]:
image_size = (180, 180)
img_height, img_width = 180, 180
batch_size = 32

print('Training dataset:')
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "data/Tobacco3482-images/train",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

print('Test dataset:')
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "data/Tobacco3482-images/test",
    labels='inferred',
    label_mode='categorical',
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Pour l'entraînement des réseaux de neurones, nous devons spécifier le nombre de classes pour la couche de prédiction.

In [ ]:
CLASSES_LIST = np.unique(data['label'])
n_out = len(CLASSES_LIST)

print(CLASSES_LIST, num_classes)

Réseau de neurones convolutifs pour la classification d'images (avec 3 couches convolutives):

In [ ]:
from tensorflow.keras import Sequential

model = Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes) # couche de prédiction
])


Nous préparons le réseau pour l'entraînement.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
print('Model')
model.summary()

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(2021).prefetch(buffer_size=AUTOTUNE)

epochs = 10 # train the model for 10 epochs
history = model.fit(
  train_dataset,
  epochs=epochs
)

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

y_test = []
y_pred = []

for image, y in test_dataset: # iterate in the test dataset
    y_pred += list(model.predict_classes(image)) # add the predictions of the batch to our list
    y_test += list(np.argmax(y.numpy(), axis=-1)) # add the true labels to our list

print(classification_report(y_test, y_pred, target_names=CLASSES_LIST))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, xticklabels=CLASSES_LIST, yticklabels=CLASSES_LIST)
plt.ylabel('True classes')
plt.xlabel('Predicted classes')

plt.show()